In [31]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt




In [32]:
df=pd.read_csv('bengaluru_house_prices.csv')

In [33]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [34]:
df['availability'].unique()

array(['19-Dec', 'Ready To Move', '18-May', '18-Feb', '18-Nov', '20-Dec',
       '17-Oct', '21-Dec', '19-Sep', '20-Sep', '18-Mar', '20-Feb',
       '18-Apr', '20-Aug', '18-Oct', '19-Mar', '17-Sep', '18-Dec',
       '17-Aug', '19-Apr', '18-Jun', '22-Dec', '22-Jan', '18-Aug',
       '19-Jan', '17-Jul', '18-Jul', '21-Jun', '20-May', '19-Aug',
       '18-Sep', '17-May', '17-Jun', '21-May', '18-Jan', '20-Mar',
       '17-Dec', '16-Mar', '19-Jun', '22-Jun', '19-Jul', '21-Feb',
       'Immediate Possession', '19-May', '17-Nov', '20-Oct', '20-Jun',
       '19-Feb', '21-Oct', '21-Jan', '17-Mar', '17-Apr', '22-May',
       '19-Oct', '21-Jul', '21-Nov', '21-Mar', '16-Dec', '22-Mar',
       '20-Jan', '21-Sep', '21-Aug', '14-Nov', '19-Nov', '15-Nov',
       '16-Jul', '15-Jun', '17-Feb', '20-Nov', '20-Jul', '16-Sep',
       '15-Oct', '15-Dec', '16-Oct', '22-Nov', '15-Aug', '17-Jan',
       '16-Nov', '20-Apr', '16-Jan', '14-Jul'], dtype=object)

In [35]:
df['society'].unique()

array(['Coomee ', 'Theanmp', nan, ..., 'SJovest', 'ThhtsV ', 'RSntsAp'],
      dtype=object)

In [36]:
len(df['society'].unique())

2689

In [37]:
df.shape

(13320, 9)

In [38]:
df2=df.drop(['area_type','availability','society','balcony'],axis=1)

In [39]:
df2.shape

(13320, 5)

### Handling Missing values

In [40]:
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [41]:
df2.dropna(inplace=True)

In [42]:
df2.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

## Feature Engineering

In [43]:
df2['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

##### add new feature is BHK

In [44]:
df2['bhk']=df2['size'].apply(lambda x:x.split(' ')[0])

In [45]:
df2.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13246 entries, 0 to 13319
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13246 non-null  object 
 1   size        13246 non-null  object 
 2   total_sqft  13246 non-null  object 
 3   bath        13246 non-null  float64
 4   price       13246 non-null  float64
 5   bhk         13246 non-null  object 
dtypes: float64(2), object(4)
memory usage: 724.4+ KB


In [47]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [48]:
#### here we want to change the data types and change the range to a value

In [49]:
def convert_sqft_to_number(X):
    tokens=X.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+float(tokens[1]))/2
    else:
        try:
            return float(tokens)
        except:
            return None
        

In [50]:
df2['total_sqft']=df2['total_sqft'].apply(convert_sqft_to_number)
df2=df2[df2.total_sqft.notnull()]

## Feature Englineering

In [51]:
df2.head()

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2475.0,4.0,186.000,4
122,Hebbal,4 BHK,5611.5,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1073.5,2.0,54.005,2
165,Sarjapur,2 BHK,1242.5,2.0,43.490,2
188,KR Puram,2 BHK,1277.5,2.0,56.800,2


In [53]:
df3=df2.copy()
df3['price_per_total_sqft']=df3['price']*100000//df3['total_sqft']

In [54]:
df3

,location,size,total_sqft,bath,price,bhk,price_per_total_sqft
30,Yelahanka,4 BHK,2475.0,4.0,186.000,4,7515.0
122,Hebbal,4 BHK,5611.5,4.0,477.000,4,8500.0
137,8th Phase JP Nagar,2 BHK,1073.5,2.0,54.005,2,5030.0
165,Sarjapur,2 BHK,1242.5,2.0,43.490,2,3500.0
188,KR Puram,2 BHK,1277.5,2.0,56.800,2,4446.0
...,...,...,...,...,...,...,...
12975,Whitefield,2 BHK,955.0,2.0,38.190,2,3998.0
12990,Talaghattapura,3 BHK,2038.5,3.0,122.000,3,5984.0
13059,Harlur,2 BHK,1335.0,2.0,72.760,2,5450.0
13265,Hoodi,2 BHK,1258.5,2.0,59.135,2,4698.0


In [56]:
df3.describe()

,total_sqft,bath,price,price_per_total_sqft
count,144.000000,144.000000,144.000000,144.000000
mean,1631.879062,2.618056,91.251979,5034.104167
std,1128.863151,1.582229,89.486900,1453.897452
min,412.500000,1.000000,13.260000,2399.000000
25%,797.625000,1.000000,36.395000,4299.000000
50%,1240.000000,2.000000,55.702500,4844.500000
75%,2096.000000,3.250000,115.222500,5444.750000
max,5667.500000,7.000000,477.000000,10998.000000
